# Introduction to p8 python client for agentic systems
- Percolate is a database that allows agents to be built entirely in the data tier
- However either for testing or to orchastrate more complex agents in code, we provide a python guide
- This giude assumes that you have followed the guides to add some initial data and tools
- to learn more see here

In [ ]:
import percolate as p8

# Agents are simple in Percolate
- They are just Pydantic objects with references to extenral tools in your function repository
- You can provide structure and you can use this structure to save and search data in the database for your agent


In [ ]:
from pydantic import BaseModel,Field
import typing
from percolate.models import DefaultEmbeddingField

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    #because it has no config it will save to the public database schema
    
    name: str = Field(description="Task name")
    #the default embedding field just settgs json_schema_extra.embedding_provider so you can do that yourself
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {}

In [ ]:
p8.Agent(MyFirstAgent).run("what is the capital of ireland")

In [ ]:
# 0.test registering api fuctions and later apis
# 1.add the functions that we have in get_funtions on register external functions in this context are added as inline functions in this approach
# 2.test other models including grok xai
# 3.external functions are searchable by plans
# 4.test inline functions even though we dont use them
# 5.test restoring a function from the database as above
# 6.test planning over entities / functions
# 7.test using native functions
# 8.add the database interface for function calls - lots of tests for different api responses
# 9.test loading entities and their functions
#10.test crud

## Behind the scenes
- we created a simple rest api wrapper instead of adding various api python clients
- you can can iterate over these check the response of each model for which you have an API key defined
- you should first check the langauge model is in your data store - either you have added the API token there or in the env var given by the reference to the env
- we will then iterate and ask the same question first with tools and then without tools to see what the payloads look like for testing
- **This is illustrated in `percolate.services.llm._check_all`**
- the high level wrapper we use parsers to a single response scheme

In [9]:
from percolate.services.llm import _check_all, LanguageModel
#some example response are in the tests

### asking without tools

In [2]:
gem = LanguageModel('gemini-1.5-flash')
gem.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:34:03.439 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model gemini-1.5-flash


AIResponse(id='fe3554ae-dcaa-11ef-889b-7606330c2360', model_name='gemini-1.5-flash', tokens=None, tokens_in=16, tokens_out=30, tokens_other=None, session_id=None, role='model', content='The capital of Ireland is Dublin.\n\nDid you know that Ireland is the only island in the world with two official languages, Irish and English?\n', status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [2]:
claude = LanguageModel('claude-3-5-sonnet-20241022')
claude.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:37:45.400 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model claude-3-5-sonnet-20241022


AIResponse(id='843d1b18-dcab-11ef-b21f-7606330c2360', model_name='claude-3-5-sonnet-20241022', tokens=None, tokens_in=23, tokens_out=106, tokens_other=None, session_id=None, role='assistant', content="The capital of Ireland is Dublin. It's both the largest city and the capital, located on the eastern coast of the country at the mouth of the River Liffey.\n\nRandom fact: While we're talking about Ireland, did you know that the submarine was invented by an Irishman? John Philip Holland, born in County Clare, Ireland, created the first submarine to be formally commissioned by the U.S. Navy in 1900. His design became the foundation for modern submarine development!", status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [8]:
xai = LanguageModel('grok-2-latest')
xai.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:42:09.993 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model grok-2-latest


AIResponse(id='208abf2a-dcac-11ef-b21f-7606330c2360', model_name='grok-2-1212', tokens=None, tokens_in=26, tokens_out=46, tokens_other=None, session_id=None, role='assistant', content='The capital of Ireland is Dublin. \n\nRandom fact: Did you know that Dublin is home to the famous Guinness Brewery, which was founded in 1759 and is one of the most visited tourist attractions in Ireland?', status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [9]:
deepseek = LanguageModel('deepseek-chat')
deepseek.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:42:40.933 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model deepseek-chat


AIResponse(id='33c8598a-dcac-11ef-b21f-7606330c2360', model_name='deepseek-chat', tokens=None, tokens_in=19, tokens_out=55, tokens_other=None, session_id=None, role='assistant', content='The capital of Ireland is Dublin. \n\nRandom fact: Dublin is home to the famous Guinness Storehouse, where you can learn about the history of Guinness beer and enjoy a pint with a panoramic view of the city from the Gravity Bar. Cheers! 🍻', status='RESPONSE', tool_calls=[], tool_eval_data=None)

### asking with tools
- we create a fictitious tool first - this tested the response in a simple way

In [1]:
from percolate.services.llm import _check_all, LanguageModel

fns =[{
  "name": "get_weather",
  "description": "Get the weather forecast for a specific city and date",
  "parameters": {
    "type": "object",
    "properties": {
      "city": {
        "type": "string",
        "description": "The city for which to get the weather forecast"
      },
      "date": {
        "type": "string",
        "description": "The date for the weather forecast (YYYY-MM-DD)"
      }
    },
    "required": ["city", "date"]
  }
}]



In [2]:
claude = LanguageModel('claude-3-5-sonnet-20241022')
claude.ask("What is the weather in dublin tomorrow", functions=None)


2025-01-27 08:39:35.873 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:169 - invoking model claude-3-5-sonnet-20241022
2025-01-27 08:39:35.874 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:240 - request data={'model': 'claude-3-5-sonnet-20241022', 'temperature': 0.01, 'messages': [{'role': 'user', 'content': 'What is the weather in dublin tomorrow'}], 'max_tokens': 8192}


AIResponse(id='2731af0c-dcb4-11ef-b42b-7606330c2360', model_name='claude-3-5-sonnet-20241022', tokens=None, tokens_in=14, tokens_out=87, tokens_other=None, session_id=None, role='assistant', content='I am not able to provide real-time weather forecasts or predictions. To get accurate weather information for Dublin tomorrow, I recommend:\n\n1. Checking a weather website or app (like Met Éireann for Ireland)\n2. Using your local weather service\n3. Checking news websites with weather sections\n\nThese sources will give you up-to-date and accurate weather forecasts for Dublin.', status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [4]:
gem = LanguageModel('gemini-1.5-flash')
gem.ask("What is the weather in dublin", functions=fns)


2025-01-27 08:39:51.748 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:169 - invoking model gemini-1.5-flash
2025-01-27 08:39:51.750 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:240 - request data={'contents': [{'role': 'user', 'parts': {'text': 'What is the weather in dublin'}}], 'tool_config': {'function_calling_config': {'mode': 'ANY'}}, 'tools': [{'function_declarations': [{'name': 'get_weather', 'description': 'Get the weather forecast for a specific city and date', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city for which to get the weather forecast'}, 'date': {'type': 'string', 'description': 'The date for the weather forecast (YYYY-MM-DD)'}}, 'required': ['city', 'date']}}]}]}


AIResponse(id='2faba750-dcb4-11ef-b42b-7606330c2360', model_name='gemini-1.5-flash', tokens=None, tokens_in=81, tokens_out=17, tokens_other=None, session_id=None, role='model', content='', status='TOOL_CALLS', tool_calls=[{'function': {'name': 'get_weather', 'arguments': {'city': 'dublin', 'date': '2024-02-29'}}}], tool_eval_data=None)

In [3]:
gmin = LanguageModel('gpt-4o-mini')
gmin.ask("What is the weather in dublin tomorrow", functions=fns)


2025-01-27 08:29:41.358 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:168 - invoking model gpt-4o-mini
2025-01-27 08:29:41.359 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:239 - request data={'model': 'gpt-4o-mini', 'messages': [{'role': 'user', 'content': 'What is the weather in dublin tomorrow'}], 'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather forecast for a specific city and date', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city for which to get the weather forecast'}, 'date': {'type': 'string', 'description': 'The date for the weather forecast (YYYY-MM-DD)'}}, 'required': ['city', 'date']}}}], 'temperature': 0.01}


AIResponse(id='c407fc98-dcb2-11ef-94f1-7606330c2360', model_name='gpt-4o-mini-2024-07-18', tokens=None, tokens_in=82, tokens_out=25, tokens_other=None, session_id=None, role='assistant', content='', status='TOOL_CALLS', tool_calls=[{'id': 'call_hZgUO0LgPAklKbtlnl4nGc0V', 'type': 'function', 'function': {'name': 'get_weather', 'arguments': '{"city":"Dublin","date":"2023-10-06"}'}}], tool_eval_data=None)

In [1]:
import percolate as p8
from pydantic import BaseModel,Field
import typing
from percolate.models import DefaultEmbeddingField

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    #because it has no config it will save to the public database schema
    
    name: str = Field(description="Task name")
    #the default embedding field just settgs json_schema_extra.embedding_provider so you can do that yourself
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_model_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {
            'get_pet_findByStatus': "a function i used to look up petes based on their status",
            'p8_About' : 'a "native" database function that gives me general information about percolate'
        }

 

In [4]:
agent = p8.Agent(MyFirstAgent,allow_help=False)
#agent.run("list two sold pets",language_model='grok-2-latest')
#agent.run("list two sold pets",language_model='deepseek-chat')
#agent.run("list two sold pets",language_model='groq-llama-3.3-70b-versatile')
#agent.run("list two sold pets",language_model='gemini-1.5-flash')
#agent.run("list two sold pets",language_model='claude-3-5-sonnet-20241022')
agent.run("list two sold pets",language_model='cerebras-llama3.1-8b')

2025-01-28 08:47:29.911 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function get_pet_findByStatus
2025-01-28 08:47:29.913 | WARNING  | percolate.services.FunctionManager:activate_agent_context:50 - We could not find the function {'p8_About'}
2025-01-28 08:47:29.913 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.get_entities of Runner(public.MyFirstAgent)>
2025-01-28 08:47:29.919 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function get_entities
2025-01-28 08:47:29.920 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.search of Runner(public.MyFirstAgent)>
2025-01-28 08:47:29.922 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function search
2025-01-28 08:47:29.923 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.activate_function

2025-01-28 08:47:31.160 | WARNING  | percolate.services.llm.LanguageModel:call_api_simple:291 - failed to submit: response.status_code=400  b'{"message":"Please reduce the length of the messages or completion. Current length is 10361 while limit is 8192","type":"invalid_request_error","param":"messages","code":"context_length_exceeded"}'
2025-01-28 08:47:31.162 | WARNING  | percolate.services.llm.LanguageModel:parse:62 - unexpected structure in OpenAI scheme message response={'message': 'Please reduce the length of the messages or completion. Current length is 10361 while limit is 8192', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'} - caused the error 'choices'
2025-01-28 08:47:31.165 | WARNING  | percolate.services.llm.LanguageModel:parse:151 - failing to parse response={'message': 'Please reduce the length of the messages or completion. Current length is 10361 while limit is 8192', 'type': 'invalid_request_error', 'param': 'messages', 'code'

AttributeError: 'NoneType' object has no attribute 'status'

In [3]:
{'model': 'deepseek-chat', 'messages': [{'role': 'system', 'content': 'You are an agent that provides the information you are asked and a second random fact\n# General Advice.\nUse whatever functions are available to you and use world knowledge only if prompted \nor if there is not other way \nor the user is obviously asking about real world information that is not covered by functions.\nObserve what functions you have to use and check the message history to avoid calling the same functions with the same parameters repeatedly.\nIf you find a function name in your search, you can activate it by naming using one of your functions. You should do so without asking the user.\n            '}, {'role': 'user', 'content': 'list two sold pets'}], 'tools': [{'type': 'function', 'function': {'name': 'get_pet_findByStatus', 'description': 'Multiple status values can be provided with comma separated strings', 'parameters': {'type': 'object', 'properties': {'status': {'type': 'array', 'description': 'Status values that need to be considered for filter', 'items': {'type': 'string', 'description': '', 'enum': ['available', 'pending', 'sold']}}}, 'required': ['status']}}}, {'type': 'function', 'function': {'name': 'get_entities', 'parameters': {'properties': {'keys': {'description': 'one or more names to use to lookup the entity or entities', 'type': 'string'}}, 'required': ['keys'], 'type': 'object'}, 'description': 'Lookup entity by one or more keys. For example if you encounter entity names or keys in question, data etc you can use\nthe entity search to learn more about them'}}, {'type': 'function', 'function': {'name': 'search', 'parameters': {'properties': {'questions': {'description': 'ask one or more questions to search the data store', 'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['questions'], 'type': 'object'}, 'description': 'Run a general search on the model that is being used in the current context as per the system prompt\nIf you want to add multiple questions supply a list of strings as an array.'}}, {'type': 'function', 'function': {'name': 'activate_functions_by_name', 'parameters': {'properties': {'function_names': {'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['function_names'], 'type': 'object'}, 'description': 'Provide a list of function names to load.\nThe names should be fully qualified object_id.function_name'}}], 'temperature': 0.01}

{'model': 'deepseek-chat',
 'messages': [{'role': 'system',
   'content': 'You are an agent that provides the information you are asked and a second random fact\n# General Advice.\nUse whatever functions are available to you and use world knowledge only if prompted \nor if there is not other way \nor the user is obviously asking about real world information that is not covered by functions.\nObserve what functions you have to use and check the message history to avoid calling the same functions with the same parameters repeatedly.\nIf you find a function name in your search, you can activate it by naming using one of your functions. You should do so without asking the user.\n            '},
  {'role': 'user', 'content': 'list two sold pets'}],
 'tools': [{'type': 'function',
   'function': {'name': 'get_pet_findByStatus',
    'description': 'Multiple status values can be provided with comma separated strings',
    'parameters': {'type': 'object',
     'properties': {'status': {'type': 

In [25]:
d = {'contents': [{'role': 'user', 'parts': {'text': 'list two sold pets'}}, {'parts': [{'functionCall': {'name': 'get_pet_findByStatus', 'args': {'status': ['sold']}}}], 'role': 'model'}, {'role': 'user', 'parts': [{'functionResponse': {'name': 'get_pet_findByStatus', 'response': {'name': 'get_pet_findByStatus', 'content': [{'id': 7574746, 'name': 'Hardy', 'photoUrls': ['https://i.pinimg.com/originals/5c/2c/44/5c2c44f16da1caf134c00bc4f9c72ea0.jpg'], 'tags': [], 'status': 'sold'}, {'id': 13, 'category': {'id': 0, 'name': 'cats'}, 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 319, 'category': {'id': 599, 'name': 'UWvlC6mSeP'}, 'name': '9Yvwc4oI2v', 'photoUrls': ['tpGoWHBlVQ', 'yeGcvQrgCk', 'qr1T285XmO', 'RgDxV4E6ee', 'QiuprryEoJ'], 'tags': [{'id': 9134, 'name': 'R5GDJbOzCD'}, {'id': 7733, 'name': 'DvT5LemiI4'}, {'id': 822, 'name': 'nIjhVxRWky'}, {'id': 7878, 'name': 'LCF0VMsPyJ'}], 'status': 'sold'}, {'id': 777666, 'category': {'id': 1, 'name': 'Котик'}, 'name': 'Барсик', 'photoUrls': ['https://ref_to_img.png'], 'tags': [{'id': 1, 'name': 'Тэг 1'}], 'status': 'sold'}, {'id': 5615, 'category': {'id': 9134, 'name': 'R5GDJbOzCD'}, 'name': 'RQ6gKb5alV', 'photoUrls': ['kDz8i5r5L6', '7Z0O9qtpGo', 'WHBlVQyeGc'], 'tags': [{'id': 7257, 'name': 'rgCkqr1T28'}, {'id': 7044, 'name': 'RgDxV4E6ee'}, {'id': 2274, 'name': 'uprryEoJvk'}], 'status': 'sold'}, {'id': 8153, 'category': {'id': 5416, 'name': 'vUJZkcGjev'}, 'name': 'Q1jM48IsJL', 'photoUrls': ['58RC38Qduf', 'wBaep9TEDg'], 'tags': [{'id': 1291, 'name': 'kgRX0bHKXc'}, {'id': 6493, 'name': 'PDDqT1qVrN'}, {'id': 6334, 'name': 'I8qaIhlzo5'}, {'id': 2360, 'name': 'H1Dr5pWtB6'}, {'id': 661, 'name': 'JQ7pZ1Vqj9'}], 'status': 'sold'}, {'id': 485, 'category': {'id': 4190, 'name': 'I2vyOHsrKK'}, 'name': 'dNDYRs7rkp', 'photoUrls': ['v2gGq5MiQt'], 'tags': [{'id': 6990, 'name': 'PyJUWvlC6m'}, {'id': 1036, 'name': 'PwbMF9Yvwc'}], 'status': 'sold'}, {'id': 188530, 'category': {'id': 90274, 'name': 'tbhBBPKt'}, 'name': 'doggie', 'photoUrls': ['SvoqWh'], 'tags': [{'id': 428371, 'name': 'RUYsQwyG'}], 'status': 'sold'}, {'id': 9815, 'category': {'id': 4869, 'name': 'WTohOTTkE'}, 'name': 'ivyFEhMAoVvGDPZfIwB', 'photoUrls': ['yGuF'], 'tags': [{'id': 5167, 'name': 'aVdpnmvXzUndaHJI'}], 'status': 'sold'}, {'id': 989, 'category': {'id': 6675, 'name': 'N1DnMMDP5G'}, 'name': 'pyX1Fz7tPQ', 'photoUrls': ['jM48IsJL45', '8RC38Qdufw', 'Baep9TEDgr'], 'tags': [{'id': 4878, 'name': 'gRX0bHKXcS'}, {'id': 947, 'name': 'DDqT1qVrN4'}, {'id': 4590, 'name': '8qaIhlzo5Q'}, {'id': 759, 'name': '1Dr5pWtB6t'}], 'status': 'sold'}, {'id': 173218104, 'category': {'id': 4, 'name': 'bird'}, 'name': 'Garibaldo', 'photoUrls': [], 'tags': [{'id': 3, 'name': 'raro'}], 'status': 'sold'}, {'id': 969732200, 'name': 'X', 'photoUrls': ['iopDYenrs'], 'tags': [], 'status': 'sold'}, {'id': 9537, 'name': 'Fluffy625', 'photoUrls': [], 'tags': [], 'status': 'sold'}, {'id': 4438, 'name': 'Fluffy833', 'photoUrls': [], 'tags': [], 'status': 'sold'}, {'id': 722337203685477354, 'name': 'Chappi dog 66', 'photoUrls': ['./dog.png'], 'tags': [], 'status': 'sold'}, {'id': 12, 'category': {'id': 0, 'name': 'dogs'}, 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 1535001, 'category': {'id': 1, 'name': 'cat'}, 'name': 'Nero', 'photoUrls': [], 'tags': [{'id': 1, 'name': 'vaccined'}], 'status': 'sold'}, {'id': 1535004, 'category': {'id': 4, 'name': 'bird'}, 'name': 'Loro', 'photoUrls': [], 'tags': [{'id': 3, 'name': 'raro'}], 'status': 'sold'}, {'id': 368991102, 'category': {'id': 1, 'name': 'dog'}, 'name': 'Snoopy', 'photoUrls': [], 'tags': [{'id': 9, 'name': 'vaccined'}], 'status': 'sold'}, {'id': 368991105, 'category': {'id': 4, 'name': 'bird'}, 'name': 'Garibaldo', 'photoUrls': [], 'tags': [{'id': 3, 'name': 'raro'}], 'status': 'sold'}, {'id': 919133, 'category': {'id': 3, 'name': 'bird'}, 'name': 'Garibaldo', 'photoUrls': [], 'tags': [{'id': 3, 'name': 'raro'}], 'status': 'sold'}, {'id': 8867, 'category': {'id': 379, 'name': 'KEACBBUq6u'}, 'name': 'BIHeP0S5zu', 'photoUrls': ['gHL2jflutf', 'RQ6gKb5alV'], 'tags': [{'id': 2676, 'name': '44cJHhL9oK'}, {'id': 3026, 'name': 'fwMr5mw4mQ'}, {'id': 5464, 'name': '30oLYMKEfI'}, {'id': 3533, 'name': '44cTrvLYEm'}, {'id': 4096, 'name': '5ytTTYQlmR'}], 'status': 'sold'}, {'id': 8623, 'category': {'id': 4161, 'name': 'vT5LemiI43'}, 'name': 'z8i5r5L67Z', 'photoUrls': ['O9qtpGoWHB', 'lVQyeGcvQr', 'gCkqr1T285'], 'tags': [{'id': 9139, 'name': 'gDxV4E6eeQ'}, {'id': 280, 'name': 'prryEoJvkq'}, {'id': 6039, 'name': '5GDJbOzCD8'}], 'status': 'sold'}, {'id': 1129, 'category': {'id': 3059, 'name': 'alVWkDz8i5'}, 'name': 'L67Z0O9qtp', 'photoUrls': ['oWHBlVQyeG', 'cvQrgCkqr1', 'T285XmORgD', 'xV4E6eeQiu'], 'tags': [{'id': 7314, 'name': 'EoJvkqAR5G'}, {'id': 5965, 'name': 'bOzCD8ZDvT'}, {'id': 4671, 'name': '5LemiI43sn'}], 'status': 'sold'}, {'id': 228, 'category': {'id': 228, 'name': 'cats'}, 'name': 'stepaa', 'photoUrls': ['string'], 'tags': [{'id': 228, 'name': 'krutoy'}], 'status': 'sold'}, {'id': 6310, 'category': {'id': 1441, 'name': 'kfvEllpbc3'}, 'name': 'w5hC0r1t7y', 'photoUrls': ['r7rLsugaq3', 'oqnxK5oj3r'], 'tags': [{'id': 8507, 'name': 'Xag5wFEqMU'}, {'id': 2689, 'name': 'blByL5lmYb'}], 'status': 'sold'}, {'id': 7708, 'category': {'id': 1244, 'name': 'Kb5alVWkDz'}, 'name': 'mQvi8a30oL', 'photoUrls': ['MKEfIGj44c', 'TrvLYEmlo5', 'ytTTYQlmRP', 'TKEACBBUq6'], 'tags': [{'id': 1529, 'name': 'IHeP0S5zuB'}, {'id': 4890, 'name': 'gHL2jflutf'}], 'status': 'sold'}, {'id': 6526, 'category': {'id': 2990, 'name': '285XmORgDx'}, 'name': 'eeQiuprryE', 'photoUrls': ['r5L67Z0O9q', 'tpGoWHBlVQ'], 'tags': [{'id': 68, 'name': 'vQrgCkqr1T'}], 'status': 'sold'}, {'id': 3145, 'category': {'id': 2645, 'name': 'pZ1Vqj9Buv'}, 'name': 'ZkcGjevRL7', 'photoUrls': ['xceBZQ8Gpq', '2P4rbjXZNi', '6D89Ispjkm', 'RGocb0VZyT', 's0E3wbsqFC'], 'tags': [{'id': 8365, 'name': 'qT1qVrN4GI'}, {'id': 759, 'name': '8qaIhlzo5Q'}, {'id': 7123, 'name': 'r5pWtB6tHJ'}], 'status': 'sold'}, {'id': 9686, 'category': {'id': 818, 'name': 'YRAjC2UxDN'}, 'name': 'RkMXvsHXQL', 'photoUrls': ['PpxXCBJfsD', 'LfhLtMZDiG', 'fPo1F6lJQs', 'jN9io0hqtW', 'nWakmPuGDt'], 'tags': [{'id': 2211, 'name': 'NLRx96nMt8'}, {'id': 9532, 'name': 'kxUZrQSQbV'}, {'id': 3107, 'name': 'WDLWPUPH0U'}], 'status': 'sold'}, {'id': 4620, 'category': {'id': 380, 'name': 'U1kJ5A4mmH'}, 'name': 's1nuq3H3LN', 'photoUrls': ['ChtA647YVX'], 'tags': [{'id': 4384, 'name': 'aXKlFvtxn1'}, {'id': 7861, 'name': 'a73N446lGH'}], 'status': 'sold'}, {'id': 3957, 'category': {'id': 6900, 'name': 'CVGcZ6MuYH'}, 'name': 'rcHLcEf99Z', 'photoUrls': ['L2qqnAdN8a', 'gHQcRIa0cq'], 'tags': [{'id': 8563, 'name': 'PrjvA5tRIl'}, {'id': 6451, 'name': 'Z33iEvDZFD'}], 'status': 'sold'}, {'id': 12438759330, 'category': {'id': 1, 'name': 'string'}, 'name': 'Musya', 'photoUrls': [], 'tags': [{'id': 1, 'name': 'string'}], 'status': 'sold'}, {'id': 3385456567326529535, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet MzcO', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 6454593534137472936, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet tS8t', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 6557351922405633463, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet obG2', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 5644743866114257558, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet ppzY', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 4691847337223564293, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet Q3Ha', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 222777289281671581, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet tQnA', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 2633791803304961804, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet l9t2', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 3880638013423558963, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet E2pm', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 7315483631429072551, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet Mr5z', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 6379402911281105569, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet zHkd', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 9152100742055004976, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet h60D', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 4310026429227574333, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet TcEr', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 5922286143638176591, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet 9sZA', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 1900497538583110533, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet JLrV', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 8910201636999394941, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet zN1Q', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 5277736444963477610, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet psC6', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 2215, 'category': {'id': 1532, 'name': 'j9BuvUJZkc'}, 'name': 'evRL7QxceB', 'photoUrls': ['aep9TEDgrG', 'lxkgRX0bHK', 'XcSRPDDqT1', 'qVrN4GI8qa'], 'tags': [{'id': 8151, 'name': 'o5QiH1Dr5p'}, {'id': 5563, 'name': 'WtB6tHJQ7p'}], 'status': 'sold'}, {'id': 4403674144605383363, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet yk0W', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 4915155796531733449, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet wpeX', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 6257391004599383370, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet coDJ', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 1733211601990494633, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet C4To', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 8248782148002652083, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet nL4H', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 8186888459947297313, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet qAW0', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 7131942443674497864, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet ciiJ', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 3449959274758994159, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet 2o3A', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}, {'id': 7204921522818020256, 'category': {'id': 0, 'name': 'Conexa Original Category'}, 'name': 'Conexa Pet uXL3', 'photoUrls': ['string'], 'tags': [{'id': 0, 'name': 'string'}], 'status': 'sold'}]}}}]}], 'tool_config': {'function_calling_config': {'mode': 'ANY'}}, 'tools': [{'function_declarations': [{'name': 'get_pet_findByStatus', 'description': 'Multiple status values can be provided with comma separated strings', 'parameters': {'type': 'object', 'properties': {'status': {'type': 'array', 'description': 'Status values that need to be considered for filter', 'items': {'type': 'string', 'description': '', 'enum': ['available', 'pending', 'sold']}}}, 'required': ['status']}}, {'name': 'get_entities', 'parameters': {'properties': {'keys': {'description': 'one or more names to use to lookup the entity or entities', 'type': 'string'}}, 'required': ['keys'], 'type': 'object'}, 'description': 'Lookup entity by one or more keys. For example if you encounter entity names or keys in question, data etc you can use\nthe entity search to learn more about them'}, {'name': 'search', 'parameters': {'properties': {'questions': {'description': 'ask one or more questions to search the data store', 'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['questions'], 'type': 'object'}, 'description': 'Run a general search on the model that is being used in the current context as per the system prompt\nIf you want to add multiple questions supply a list of strings as an array.'}, {'name': 'activate_functions_by_name', 'parameters': {'properties': {'function_names': {'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['function_names'], 'type': 'object'}, 'description': 'Provide a list of function names to load.\nThe names should be fully qualified object_id.function_name'}]}], 'system_instruction': {'parts': {'text': 'You are an agent that provides the information you are asked and a second random fact\n# General Advice.\nUse whatever functions are available to you and use world knowledge only if prompted \nor if there is not other way \nor the user is obviously asking about real world information that is not covered by functions.\nObserve what functions you have to use and check the message history to avoid calling the same functions with the same parameters repeatedly.\nIf you find a function name in your search, you can activate it by naming using one of your functions. You should do so without asking the user.\n            '}}}
d

{'contents': [{'role': 'user', 'parts': {'text': 'list two sold pets'}},
  {'parts': [{'functionCall': {'name': 'get_pet_findByStatus',
      'args': {'status': ['sold']}}}],
   'role': 'model'},
  {'role': 'user',
   'parts': [{'functionResponse': {'name': 'get_pet_findByStatus',
      'response': {'name': 'get_pet_findByStatus',
       'content': [{'id': 7574746,
         'name': 'Hardy',
         'photoUrls': ['https://i.pinimg.com/originals/5c/2c/44/5c2c44f16da1caf134c00bc4f9c72ea0.jpg'],
         'tags': [],
         'status': 'sold'},
        {'id': 13,
         'category': {'id': 0, 'name': 'cats'},
         'photoUrls': ['string'],
         'tags': [{'id': 0, 'name': 'string'}],
         'status': 'sold'},
        {'id': 319,
         'category': {'id': 599, 'name': 'UWvlC6mSeP'},
         'name': '9Yvwc4oI2v',
         'photoUrls': ['tpGoWHBlVQ',
          'yeGcvQrgCk',
          'qr1T285XmO',
          'RgDxV4E6ee',
          'QiuprryEoJ'],
         'tags': [{'id': 9134, 'nam

In [47]:
tools = [{'function_declarations': [{'name': 'get_pet_findByStatus',
    'description': 'Multiple status values can be provided with comma separated strings',
    'parameters': {'type': 'object',
     'properties': {'status': {'type': 'array',
       'description': 'Status values that need to be considered for filter',
       'items': {'type': 'string',
        'description': '',
        'enum': ['available', 'pending', 'sold']}}},
     'required': ['status']}},
   {'name': 'get_entities',
    'parameters': {'properties': {'keys': {'description': 'one or more names to use to lookup the entity or entities',
       'type': 'string'}},
     'required': ['keys'],
     'type': 'object'},
    'description': 'Lookup entity by one or more keys. For example if you encounter entity names or keys in question, data etc you can use\nthe entity search to learn more about them'},
   {'name': 'search',
    'parameters': {'properties': {'questions': {'description': 'ask one or more questions to search the data store',
       'items': {'type': 'string'},
       'type': 'array'}},
     'required': ['questions'],
     'type': 'object'},
    'description': 'Run a general search on the model that is being used in the current context as per the system prompt\nIf you want to add multiple questions supply a list of strings as an array.'},
   {'name': 'activate_functions_by_name',
    'parameters': {'properties': {'function_names': {'items': {'type': 'string'},
       'type': 'array'}},
     'required': ['function_names'],
     'type': 'object'},
    'description': 'Provide a list of function names to load.\nThe names should be fully qualified object_id.function_name'}]}]
pl = {
  "contents": [
    {
      "role": "user",
      "parts": [{
        "text": "list two sold pets"
      }]
    },
    {
      "role": "model",
      "parts": [
        {
          "functionCall": {
            "name": "get_pet_findByStatus",
            "args": {
              "status": "sold",
              "limit": 2
            }
          }
        }
      ]
    },
    {
      "role": "user",
      "parts": [
        {
          "functionResponse": {
            "name": "get_pet_findByStatus",
            "response": {
              "name": "get_pet_findByStatus",
              "content": """[
                {
                  "id": 7574746,
                  "name": "Hardy",
                  "photoUrls": ["https://i.pinimg.com/originals/5c/2c/44/5c2c44f16da1caf134c00bc4f9c72ea0.jpg"],
                  "tags": [],
                  "status": "sold"
                },
                {
                  "id": 13,
                  "name": "Fluffy",
                  "photoUrls": ["https://example.com/photo.jpg"],
                  "tags": [{"id": 0, "name": "string"}],
                  "status": "sold"
                },
                   {
                  "id": 17,
                  "name": "Jack",
                  "photoUrls": ["https://example.com/photo.jpg"],
                  "tags": [{"id": 0, "name": "string"}],
                  "status": "sold"
                }
              ]"""
            }
          }
        }
      ]
    }
  ]
}

data = {
    "contents": pl['contents'][0],
    "tool_config": {
        #we could disable this for empty tools
        "function_calling_config": {"mode": "ANY"}
    },
    "tools": tools
    #'temperature': temperature
}

import os
url = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent'
url = f"{url}?key={os.environ['GEMINI_API_KEY']}"

import requests
import json
headers = {
            "Content-Type": "application/json",
        }
response = requests.get(url, headers=headers, data=json.dumps(d))
response.content

b'<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 400 (Bad Request)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border-image:

In [49]:
partial = pl['contents'][0:1]
partial

[{'role': 'user', 'parts': [{'text': 'list two sold pets'}]}]

In [50]:
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={os.environ.get('GEMINI_API_KEY')}"
headers = {
    "Content-Type": "application/json"
}
data = {
    "contents": partial,
    "tool_config": {
      "function_calling_config": {"mode": "ANY"}
    },
    "tools": tools
}
r = requests.post(url, headers=headers, data=json.dumps(data))
r.json()

{'candidates': [{'content': {'parts': [{'functionCall': {'name': 'get_pet_findByStatus',
       'args': {'status': ['sold']}}}],
    'role': 'model'},
   'finishReason': 'STOP',
   'avgLogprobs': -0.004758656490594149}],
 'usageMetadata': {'promptTokenCount': 288,
  'candidatesTokenCount': 8,
  'totalTokenCount': 296,
  'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 171}],
  'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 8}]},
 'modelVersion': 'gemini-1.5-flash'}

In [6]:
d ={'contents': [{'role': 'user', 'parts': {'text': 'are there any sold pets'}}, {'role': 'model', 'parts': {'text': {'parts': [{'functionCall': {'name': 'get_pet_findByStatus', 'args': {'status': ['sold']}}}], 'role': 'model'}}}, {'role': 'user', 'parts': {'text': {'role': 'user', 'content': '[{"id": 7574746, "category": {"id": 1, "name": "Cat"}, "name": "Hardy", "photoUrls": ["https://i.pinimg.com/originals/5c/2c/44/5c2c44f16da1caf134c00bc4f9c72ea0.jpg"], "tags": [{"id": 7805, "name": "Mainecoon"}], "status": "sold"}, {"id": 13, "category": {"id": 0, "name": "cats"}, "photoUrls": ["string"], "tags": [{"id": 0, "name": "string"}], "status": "sold"}, {"id": 777666, "category": {"id": 1, "name": "\\u041a\\u043e\\u0442\\u0438\\u043a"}, "name": "\\u0411\\u0430\\u0440\\u0441\\u0438\\u043a", "photoUrls": ["https://ref_to_img.png"], "tags": [{"id": 1, "name": "\\u0422\\u044d\\u0433 1"}], "status": "sold"}, {"id": 5615, "category": {"id": 9134, "name": "R5GDJbOzCD"}, "name": "RQ6gKb5alV", "photoUrls": ["kDz8i5r5L6", "7Z0O9qtpGo", "WHBlVQyeGc"], "tags": [{"id": 7257, "name": "rgCkqr1T28"}, {"id": 7044, "name": "RgDxV4E6ee"}, {"id": 2274, "name": "uprryEoJvk"}], "status": "sold"}, {"id": 8153, "category": {"id": 5416, "name": "vUJZkcGjev"}, "name": "Q1jM48IsJL", "photoUrls": ["58RC38Qduf", "wBaep9TEDg"], "tags": [{"id": 1291, "name": "kgRX0bHKXc"}, {"id": 6493, "name": "PDDqT1qVrN"}, {"id": 6334, "name": "I8qaIhlzo5"}, {"id": 2360, "name": "H1Dr5pWtB6"}, {"id": 661, "name": "JQ7pZ1Vqj9"}], "status": "sold"}, {"id": 485, "category": {"id": 4190, "name": "I2vyOHsrKK"}, "name": "dNDYRs7rkp", "photoUrls": ["v2gGq5MiQt"], "tags": [{"id": 6990, "name": "PyJUWvlC6m"}, {"id": 1036, "name": "PwbMF9Yvwc"}], "status": "sold"}, {"id": 188530, "category": {"id": 90274, "name": "tbhBBPKt"}, "name": "doggie", "photoUrls": ["SvoqWh"], "tags": [{"id": 428371, "name": "RUYsQwyG"}], "status": "sold"}, {"id": 9815, "category": {"id": 4869, "name": "WTohOTTkE"}, "name": "ivyFEhMAoVvGDPZfIwB", "photoUrls": ["yGuF"], "tags": [{"id": 5167, "name": "aVdpnmvXzUndaHJI"}], "status": "sold"}, {"id": 989, "category": {"id": 6675, "name": "N1DnMMDP5G"}, "name": "pyX1Fz7tPQ", "photoUrls": ["jM48IsJL45", "8RC38Qdufw", "Baep9TEDgr"], "tags": [{"id": 4878, "name": "gRX0bHKXcS"}, {"id": 947, "name": "DDqT1qVrN4"}, {"id": 4590, "name": "8qaIhlzo5Q"}, {"id": 759, "name": "1Dr5pWtB6t"}], "status": "sold"}, {"id": 173218104, "category": {"id": 4, "name": "bird"}, "name": "Garibaldo", "photoUrls": [], "tags": [{"id": 3, "name": "raro"}], "status": "sold"}, {"id": 969732200, "name": "X", "photoUrls": ["iopDYenrs"], "tags": [], "status": "sold"}, {"id": 9537, "name": "Fluffy625", "photoUrls": [], "tags": [], "status": "sold"}, {"id": 4438, "name": "Fluffy833", "photoUrls": [], "tags": [], "status": "sold"}, {"id": 722337203685477354, "name": "Chappi dog 66", "photoUrls": ["./dog.png"], "tags": [], "status": "sold"}, {"id": 1535001, "category": {"id": 1, "name": "cat"}, "name": "Nero", "photoUrls": [], "tags": [{"id": 1, "name": "vaccined"}], "status": "sold"}, {"id": 1535004, "category": {"id": 4, "name": "bird"}, "name": "Loro", "photoUrls": [], "tags": [{"id": 3, "name": "raro"}], "status": "sold"}, {"id": 368991102, "category": {"id": 1, "name": "dog"}, "name": "Snoopy", "photoUrls": [], "tags": [{"id": 9, "name": "vaccined"}], "status": "sold"}, {"id": 368991105, "category": {"id": 4, "name": "bird"}, "name": "Garibaldo", "photoUrls": [], "tags": [{"id": 3, "name": "raro"}], "status": "sold"}, {"id": 919133, "category": {"id": 3, "name": "bird"}, "name": "Garibaldo", "photoUrls": [], "tags": [{"id": 3, "name": "raro"}], "status": "sold"}, {"id": 8867, "category": {"id": 379, "name": "KEACBBUq6u"}, "name": "BIHeP0S5zu", "photoUrls": ["gHL2jflutf", "RQ6gKb5alV"], "tags": [{"id": 2676, "name": "44cJHhL9oK"}, {"id": 3026, "name": "fwMr5mw4mQ"}, {"id": 5464, "name": "30oLYMKEfI"}, {"id": 3533, "name": "44cTrvLYEm"}, {"id": 4096, "name": "5ytTTYQlmR"}], "status": "sold"}, {"id": 8623, "category": {"id": 4161, "name": "vT5LemiI43"}, "name": "z8i5r5L67Z", "photoUrls": ["O9qtpGoWHB", "lVQyeGcvQr", "gCkqr1T285"], "tags": [{"id": 9139, "name": "gDxV4E6eeQ"}, {"id": 280, "name": "prryEoJvkq"}, {"id": 6039, "name": "5GDJbOzCD8"}], "status": "sold"}]', 'tool_call_id': 'get_pet_findByStatus'}}}], 'tool_config': {'function_calling_config': {'mode': 'ANY'}}, 'tools': [{'function_declarations': [{'name': 'help', 'parameters': {'properties': {'questions': {'description': 'provide detailed questions to guide the planner. you should augment the users question with the additional context that you have e.g. a known source', 'type': 'string'}, 'context': {'description': 'any added context e.g. what tool, function, source the user or system suggests may know the answer', 'type': 'string'}}, 'required': ['questions'], 'type': 'object'}, 'description': 'If you are stuck ask for help with very detailed questions to help the planner find resources for you.\nIf you have a hint about what the source or tool to use hint that in each question that you ask.\nFor example, you can either just ask a question or you can ask "according to resource X" and then ask the question. This is important context.'}, {'name': 'get_pet_findByStatus', 'description': 'Multiple status values can be provided with comma separated strings', 'parameters': {'type': 'object', 'properties': {'status': {'type': 'array', 'description': 'Status values that need to be considered for filter', 'items': {'type': 'string', 'description': '', 'enum': ['available', 'pending', 'sold']}}}, 'required': ['status']}}, {'name': 'get_entities', 'parameters': {'properties': {'keys': {'description': 'one or more names to use to lookup the entity or entities', 'type': 'string'}}, 'required': ['keys'], 'type': 'object'}, 'description': 'Lookup entity by one or more keys. For example if you encounter entity names or keys in question, data etc you can use\nthe entity search to learn more about them'}, {'name': 'search', 'parameters': {'properties': {'questions': {'description': 'ask one or more questions to search the data store', 'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['questions'], 'type': 'object'}, 'description': 'Run a general search on the model that is being used in the current context as per the system prompt\nIf you want to add multiple questions supply a list of strings as an array.'}, {'name': 'activate_functions_by_name', 'parameters': {'properties': {'function_names': {'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['function_names'], 'type': 'object'}, 'description': 'Provide a list of function names to load.\nThe names should be fully qualified object_id.function_name'}]}], 'system_instruction': {'parts': {'text': 'You are an agent that provides the information you are asked and a second random fact\n# General Advice.\nUse whatever functions are available to you and use world knowledge only if prompted \nor if there is not other way \nor the user is obviously asking about real world information that is not covered by functions.\nObserve what functions you have to use and check the message history to avoid calling the same functions with the same parameters repeatedly.\nIf you find a function name in your search, you can activate it by naming using one of your functions. You should do so without asking the user.\n            '}}}
d['contents']

[{'role': 'user', 'parts': {'text': 'are there any sold pets'}},
 {'role': 'model',
  'parts': {'text': {'parts': [{'functionCall': {'name': 'get_pet_findByStatus',
       'args': {'status': ['sold']}}}],
    'role': 'model'}}},
 {'role': 'user',
  'parts': {'text': {'role': 'user',
    'content': '[{"id": 7574746, "category": {"id": 1, "name": "Cat"}, "name": "Hardy", "photoUrls": ["https://i.pinimg.com/originals/5c/2c/44/5c2c44f16da1caf134c00bc4f9c72ea0.jpg"], "tags": [{"id": 7805, "name": "Mainecoon"}], "status": "sold"}, {"id": 13, "category": {"id": 0, "name": "cats"}, "photoUrls": ["string"], "tags": [{"id": 0, "name": "string"}], "status": "sold"}, {"id": 777666, "category": {"id": 1, "name": "\\u041a\\u043e\\u0442\\u0438\\u043a"}, "name": "\\u0411\\u0430\\u0440\\u0441\\u0438\\u043a", "photoUrls": ["https://ref_to_img.png"], "tags": [{"id": 1, "name": "\\u0422\\u044d\\u0433 1"}], "status": "sold"}, {"id": 5615, "category": {"id": 9134, "name": "R5GDJbOzCD"}, "name": "RQ6gKb5alV"

In [7]:
agent.functions['help'].function_spec

{'name': 'help',
 'parameters': {'properties': {'questions': {'anyOf': [{'type': 'string'},
     {'items': {'type': 'string'}, 'type': 'array'}],
    'description': 'provide detailed questions to guide the planner. you should augment the users question with the additional context that you have e.g. a known source',
    'title': 'Questions'},
   'context': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'default': None,
    'description': 'any added context e.g. what tool, function, source the user or system suggests may know the answer',
    'title': 'Context'}},
  'required': ['questions'],
  'type': 'object'},
 'description': 'If you are stuck ask for help with very detailed questions to help the planner find resources for you.\nIf you have a hint about what the source or tool to use hint that in each question that you ask.\nFor example, you can either just ask a question or you can ask "according to resource X" and then ask the question. This is important context.'}

In [3]:
#b'Failed to deserialize the JSON body into the target type: messages[2].role: unknown variant `function`, expected one of `system`, `user`, `assistant`, `tool` at line 1 column 775'
agent = p8.Agent(MyFirstAgent)
agent.run("are there any sold pets")

2025-01-27 22:02:25.741 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.help of Runner(public.MyFirstAgent)>
2025-01-27 22:02:25.745 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function help
2025-01-27 22:02:25.775 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function get_pet_findByStatus
2025-01-27 22:02:25.776 | WARNING  | percolate.services.FunctionManager:activate_agent_context:50 - We could not find the function {'p8_About'}
2025-01-27 22:02:25.777 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.get_entities of Runner(public.MyFirstAgent)>
2025-01-27 22:02:25.785 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function get_entities
2025-01-27 22:02:25.786 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.search of Runner(publ

2025-01-27 22:02:32.181 | DEBUG    | percolate.services.llm.LanguageModel:__call__:156 - Response of type RESPONSE with token consumption 1125


'Yes, there are several pets that have been sold. Here are a few examples:\n\n1. **Hardy** - A Mainecoon cat.\n   ![Hardy](https://i.pinimg.com/originals/5c/2c/44/5c2c44f16da1caf134c00bc4f9c72ea0.jpg)\n\n2. A cat with unspecified details.\n   ![Cat](https://petstore3.swagger.io/resources/photos/623389095.jpg)\n\n3. A pet named "Special_char_owner_!@#$^&()`.testing".\n   ![Special_char_owner](https://petstore3.swagger.io/resources/photos/623389095.jpg)\n\n4. A pet with a name containing special characters: "见响见响仿佛!$%&()*+,-ƄƅƆḞḟṀʶʷʸăĄą".\n   ![Special Character Pet](https://petstore3.swagger.io/resources/photos/623389095.jpg)\n\nRandom Fact: The Maine Coon is one of the largest domesticated cat breeds, known for its friendly and sociable nature.'

In [3]:
agent = p8.Agent(MyFirstAgent)
agent.run("are there any avalable pets")

2025-01-27 20:23:25.129 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.help of Runner(public.MyFirstAgent)>
2025-01-27 20:23:25.138 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function help
2025-01-27 20:23:25.177 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function get_pet_findByStatus
2025-01-27 20:23:25.178 | WARNING  | percolate.services.FunctionManager:activate_agent_context:50 - We could not find the function {'p8_About'}
2025-01-27 20:23:25.179 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.get_entities of Runner(public.MyFirstAgent)>
2025-01-27 20:23:25.181 | DEBUG    | percolate.services.FunctionManager:add_function:34 - added function get_entities
2025-01-27 20:23:25.184 | DEBUG    | percolate.services.FunctionManager:add_function:30 - adding function: <bound method ModelRunner.search of Runner(publ

"Yes, there are several available pets. Here are a few examples:\n\n1. **Doggie** - A dog with ID 9222968140497180615.\n2. **Fish** - A fish with ID 9222968140497180629.\n3. **Puff** - A pet with ID 9222968140497180637.\n4. **Leo** - A German Shepherd dog with ID 9222968140497181119.\n\nIf you're interested in any specific type of pet, let me know!\n\n**Random Fact:** Did you know that the world's oldest known breed of domesticated dog is the Saluki, which dates back to ancient Egypt around 329 BC?"

# we register APIs as follows

In [1]:
from percolate.utils.ingestion import add 
add.add_api('swagger_test', 'https://petstore.swagger.io/v2/swagger.json', verbs='get')

2025-01-28 06:29:12.135 | INFO     | percolate.utils.ingestion.add:add_api:31 - Adding API uri='https://petstore.swagger.io/v2/swagger.json'
2025-01-28 06:29:12.506 | DEBUG    | percolate.utils.ingestion.add:add_api:37 - Added api uri='https://petstore.swagger.io/v2/swagger.json'


In [ ]:
# we can check the function is there

In [ ]:
from percolate.models.p8 import Function

p8.repository(Function).get_by_name(['get_pet_findByStatus'], as_model=True)

In [ ]:
#using
agent = p8.Agent(MyFirstAgent)
agent.run("can you find available pets",limit=2)

In [ ]:
#using native functions -> p8_About
p8.Agent(MyFirstAgent).run("what can you tell me about percolate")